## Example of generating moving object observations, using `MoObs` class.

This is the class that should be used to generate moving object observations. (it's beta, but it's better than early prototype). 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from moObs import MoObs

Expanded version (I want to test the timing). For a very compact way to generate observations, see the `runMoObs` function in moo.py.

Set up and read the orbits.

In [2]:
#orbitfile = 'GreenstreetPHAs.des'
#orbitfile = 'pha20141031.des'
orbitfile = 'test.des'
moo = MoObs()
moo.readOrbits(orbitfile)

In [3]:
# See what we've got in the orbit data.
print moo.orbits.columns.values
moo.orbits.head()

['H' 'M' 'a' 'argPeri' 'e' 'epoch' 'g' 'inc' 'node' 'objId' 'q' 'tPeri']


,H,M,a,argPeri,e,epoch,g,inc,node,objId,q,tPeri
0,20,7927.899625,0.102095,31.358208,0.826961,49353.16,0.15,22.827817,88.024464,0,0.186523,49090.767296
1,20,731.578994,0.619852,276.868840,0.335431,49353.16,0.15,13.337659,337.219357,1,0.827770,48990.932835
2,20,419.157169,0.414563,285.875200,0.560063,49353.16,0.15,6.353077,35.696938,2,0.646745,49239.645606
3,20,2050.066328,0.376427,267.803013,0.650216,49353.16,0.15,39.828694,356.920252,3,0.621186,48872.789084
4,20,5874.569717,0.251116,43.240574,0.763736,49353.16,0.15,1.330878,349.850231,4,0.442902,48603.137197


Generate ephemerides for ten years, with spacing of 2 hours, then generate interpolation functions.

In [4]:
# timesteps of 2 hours for NEOS; 5 hours for MBAS and 2 days for TNOs seems pretty good
moo.setTimes(timestep = 2./24., ndays=365*10, timestart = 49353.0)
moo.setupOorb()

In [5]:
%%timeit
for i, sso in moo.orbits.head(3).iterrows():
    ephs = moo.generateEphs(sso)
    interpfuncs = moo.interpolateEphs(ephs)

1 loops, best of 3: 28.6 s per loop


Getting 21.4s/loop (3 objects) and 24.8s/loop (3 objects), and 29s /3 on my laptop. This is comparable to previous estimates, where it was 8s/object for ten years. For objects more distant than NEOs, we can increase the timestep. 

As seen in the [Ephemeris Generation](./Ephemeris Generation.ipynb) notebook, the ephemeris accuracy for 2 hour timespacing + linear interpolation for NEOs is generally better than 3e-5 degrees (0.1") in position, and 8.3e-6 ("/s) in velocity. The solar elongation calculation for some NEOs seems to be faulty -- or at least, not agree with opsim (and I doubt opsim is really observing things with very low solar elongation of <20deg). That needs a little more investigation, but for now - use opsim's solar elongation values.

---

Add in actual simdata series of observations and matching moving object to the fov.

In [5]:
from lsst.sims.maf.db import OpsimDatabase

opsdb = OpsimDatabase('/Users/lynnej/myopsim/db/enigma_1189_sqlite.db')

# What columns from opsim do you want to have available when doing later 'metric' generation?
dbcols = ['expMJD', 'night', 'fieldRA', 'fieldDec', 'rotSkyPos', 'filter',
          'finSeeing', 'fiveSigmaDepth', 'visitExpTime', 'solarElong']
simdata = opsdb.fetchMetricData(dbcols, sqlconstraint='')

/Users/lynnej/anaconda/lib/python2.7/site-packages/sqlalchemy/sql/elements.py:3758: SAWarning: Textual SQL expression '' should be explicitly declared as text('') (this warning may be suppressed after 10 occurrences)
  {"expr": util.ellipses_string(element)})


In [7]:
#%%timeit
outfileName = 'test_out.txt'
try:
    del moo.outfile
except AttributeError:
    pass
useCamera=True
# Limit this example to only calculating positions for the first few objects.
for i, sso in moo.orbits.iterrows():
    ephs = moo.generateEphs(sso)
    interpfuncs = moo.interpolateEphs(ephs)
    # This is how you change the matching against the FOV. 
    #  If useCamera=True, then rFov is irrelevant - the silicon footprint is used.
    #  If useCamera=False, then rFov is used instead, as a circular footprint. 
    idxObs = moo.ssoInFov(interpfuncs, simdata, rFov=np.radians(1.75), useCamera=useCamera)
    moo.writeObs(sso['objId'], interpfuncs, simdata, idxObs, outfileName=outfileName)

With the camera on, writing to disk (desktop = SSD), this took 30s for 3 objects. For 10 objects, it was 2min+15s for the GS PHAs; for the 'phas' PHAs, only 1:45. With camera off, this took 24s for 3 objects. 

In [8]:
# See what this output file looks like. 
# This file stores the moving object observations - even if the object would have been too faint!
!head test_out.txt

objId delta ra dec magV time dradt ddecdt phase solarelon velocity obsHistID expMJD night fieldRA fieldDec rotSkyPos filter finSeeing fiveSigmaDepth visitExpTime solarElong magFilter dmagColor dmagTrail dmagDetect 
0.0 0.407789951521 178.554053942 7.85368879567 20.9694487512 49540.020904 2.52860798778 -1.7485783854 85.5255554595 70.905500509 3.07431107273 143217 49540.020904 187 3.141593 0.128524 4.063771 i 1.030507 23.394825 30.0 72.496338 20.6780969655 -0.291351785733 0.593583381506 1.04387369879 
0.0 0.407799676903 178.557464517 7.85135351862 20.9694229 49540.02224 2.52867446802 -1.74853616271 85.5223530747 70.9082229766 3.07434176401 143220 49540.02224 187 3.087533 0.127822 4.095892 i 0.955558 23.445884 30.0 69.651283 20.6780711143 -0.291351785733 0.626285340723 1.11462854188 
0.0 0.535782846221 200.57039727 -7.46103903157 21.1868442003 49551.046682 1.48019633703 -1.01948682577 64.9531469912 86.5175503045 1.79731591335 155017 49551.046682 198 3.499825 -0.114112 4.46024 z 1.015369 2